In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 1, 11)
init_page = 1
init_offset = 0

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-03-14
page  1
https://index.sindonews.com/index/0/0?t=2018-03-14
https://international.sindonews.com/read/1289739/40/myanmar-ragukan-kredibilitas-laporan-pelanggaran-ham-rohingya-pbb-1521046629
Insert berita  Myanmar Ragukan Kredibilitas Laporan Pelanggaran HAM Rohingya PBB
masuk
https://daerah.sindonews.com/read/1289736/190/4-pasangan-calon-gubernur-sumsel-fokus-pemerataan-pembangunan-1521045253
Insert berita  4 Pasangan Calon Gubernur Sumsel Fokus Pemerataan Pembangunan
masuk
https://metro.sindonews.com/read/1289711/171/penindakan-lemah-tunggakan-sewa-rusun-capai-rp35-miliar-1521038768
Insert berita  Penindakan Lemah, Tunggakan Sewa Rusun Capai Rp35 Miliar
masuk
https://soccer.sindonews.com/read/1289744/57/susunan-pemain-besiktas-vs-bayern-muenchen-1521048651
Insert berita  Susunan Pemain Besiktas vs Bayern Muenchen
masuk
